Inicialização do navegador

In [1]:
# # criar um navegador
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# import pandas as pd

# # criar o navegador
# servico = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=servico)

# # importar/visualizar a base de dados
# tabela_produtos = pd.read_excel("buscas.xlsx")
# display(tabela_produtos)




# criar um navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd

# criar o navegador
servico = Service(r'C:\Users\mathe\Downloads\chromedriver_win32\chromedriver.exe')
driver = webdriver.Chrome(service=servico)

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,2000,2800


Definição das funções de busca no google e no buscape

In [2]:
import time

def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    # verificação do nome - se no nome tem algum termo banido
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome: 
            tem_termos_banidos = True
            break
    return tem_termos_banidos


def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    # verificar se no nome tem TODOS os termos do nome do produto
    tem_todos_termos_produto = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False
            break
    return tem_todos_termos_produto


def busca_google_shopping(driver, produto, termos_banidos, preco_min, preco_max):
    # entrar no google
    driver.get("https://www.google.com/")

    # pesquisar o nome do produto no google
    driver.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

    # tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_max = float(preco_max)
    preco_min = float(preco_min)


    # clicar na aba shopping
    try:
        posicao = driver.find_element(By.XPATH, '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a')
        posicao.click()
    except:
        driver.find_element(By.XPATH, '//*[@id="cnt"]/div[5]/div/div/div/div[1]/div/a[1]').click()

    # pegar a lista de resultados da busca no google shopping
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'i0X6df')

    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
    lista_ofertas = []
    for resultado in lista_resultados:
        try: # Caso exista algum preço escrito de forma estranha que não tratamos
            nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
            nome = nome.lower()

            # verificação do nome - se no nome tem algum termo banido
            tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

            # verificar se no nome tem TODOS os termos do nome do produto
            tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

            # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produto = True
            if not tem_termos_banidos and tem_todos_termos_produto:
                # pegando preco e tratando para um float
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace("+ impostos", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)

                # pegando link
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..') # pega elemento antes do referência
                link = elemento_pai.get_attribute('href')

                # se o preço ta entre preco_min e preco_max
                if preco_min <= preco <= preco_max:
                    lista_ofertas.append((nome, preco, link))
        except:
            continue

    return lista_ofertas


def busca_buscape(driver, produto, termos_banidos, preco_min, preco_max):
    # tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_max = float(preco_max)
    preco_min = float(preco_min)

    # buscar no buscape
    driver.get('https://www.buscape.com.br/')
    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # pegar os resultados
    while len(driver.find_elements(By.CLASS_NAME, 'Select_Select__1S7HV')) < 1: # esperando entrar na página de resultados
        time.sleep(1)
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb') # pegando so resultados

    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
    lista_ofertas = []
    for resultado in lista_resultados:
        try: # Caso exista algum preço escrito de forma estranha que não tratamos
            nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
            nome = nome.lower()

            # verificação do nome - se no nome tem algum termo banido
            tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

            # verificar se no nome tem TODOS os termos do nome do produto
            tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

            # analisar se o preço está entre preço minimo e preço máximo
            if not tem_termos_banidos and tem_todos_termos_produto:
                # pegando preco e tratando para um float
                preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
                preco = preco.replace("R$", "").replace("+ impostos", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)

                # pegando link
                link = resultado.get_attribute('href')

                if preco_min <= preco <= preco_max:
                    lista_ofertas.append((nome, preco, link))

            # retortar a lista de ofertas do buscapé
        except:
            continue
    
    return lista_ofertas

Contrução da nossa lista de ofertas encontradas

In [3]:
from pathlib import Path

# Criando a pasta de Ofertas Encontradas caso ela não exista ainda
if not Path('Ofertas Encontradas').exists():
    caminho = Path.cwd() / 'Ofertas Encontradas'
    caminho.mkdir()

tabela_ofertas = pd.DataFrame() # criando tabela vazia

for linha in tabela_produtos.index:
    # Tabela do produto atual
    tabela_produto_atual = pd.DataFrame()

    # pesquisar pelo produto
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_min = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_max = tabela_produtos.loc[linha, 'Preço máximo']

    # pesquisando no google shopping
    lista_ofertas_google_shopping = busca_google_shopping(driver, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preco', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
        tabela_produto_atual = pd.concat([tabela_produto_atual, tabela_google_shopping])
    else:
        tabela_google_shopping = None

    # pesquisando no google buscapé
    lista_ofertas_buscape = busca_buscape(driver, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preco', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
        tabela_produto_atual = pd.concat([tabela_produto_atual, tabela_buscape])
    else:
        tabela_buscape = None
    
    # Criando um arquivo excel para cada produto específico
    if len(tabela_produto_atual.index) > 0:
        # Ordenando a tabela pelo preço
        tabela_produto_atual = tabela_produto_atual.sort_values(by='Preco') #
        # criando o arquivo
        produto = produto.replace(" ", "_")
        tabela_produto_atual.to_excel(f'Ofertas Encontradas/{produto}.xlsx', index=False)
    
# Criando um arquivo excel COM TODOS PRODUTOS de uma vez
display(tabela_ofertas)

,Produto,Preco,Link
0,"vitrine: iphone 12 apple 64gb azul tela 6,1 câ...",3369.00,https://www.google.com/url?url=https://www.car...
1,"vitrine: iphone 12 apple 64gb branco tela 6,1 ...",3369.00,https://www.google.com/url?url=https://www.car...
2,"vitrine: iphone 12 apple 64gb roxo tela 6,1 câ...",3369.00,https://www.google.com/url?url=https://www.car...
3,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",3359.00,https://www.google.com/url?url=https://www.car...
4,vitrine apple iphone 12 64gb original com gara...,3159.00,https://www.google.com/url?url=https://produto...
...,...,...,...
7,placa de vídeo rtx 3060 ventus 2x msi nvidia g...,2198.99,https://www.buscape.com.br/lead?oid=937934447&...
8,placa de vídeo nvidia msi rtx 3060 8gb gddr6 v...,2138.44,https://www.buscape.com.br/lead?oid=1061512137...
9,placa de vídeo rtx 3060 oc edition asus dual g...,2059.99,https://www.buscape.com.br/lead?oid=1005057132...
10,placa de video rtx 3060 gaming oc gigabyte gef...,2399.99,https://www.buscape.com.br/lead?oid=453950717&...


Exportar a base de ofertas para Excel

In [4]:
# exportar pro excel (tabela com todos produtos)
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

Enviando o e-mail

In [5]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'matheusporn7@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>A tabela de com itens juntos, e a de todos itens separados também será enviada com documentos Excel</p>
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    <p>Matheus</p>
    """
    attachment = Path.cwd() / 'Ofertas.xlsx'
    mail.Attachments.Add(str(attachment))

    caminho = Path.cwd() / "Ofertas Encontradas"
    arquivos_ofertas_encontradas = caminho.iterdir()
    for arquivo in arquivos_ofertas_encontradas:
        mail.Attachments.Add(str(Path(arquivo)))

    mail.Send()

driver.quit()